# Notebook 2: Physics Simulation and Robot Loading

**Learning Objectives:**
1. Load real robots into Drake using `Parser` and pre-specified robot files (URDF, SDF, MJCF)
2. Understand Drake's physics simulation engine (`MultibodyPlant`) and the geometry engine (`SceneGraph`)
3.  Create custom 3D assets (your initials!) and learn `URDF` and `SDF` authoring
4. Visualize robots, objects, and simulations using Meshcat
5. Build complete robot simulations with custom objects in Drake

**What you'll build:** A complete simulation featuring the IIWA14 robot arm interacting with your custom initials asset!

---


## Setup and Imports

Let us first import Drake functionality for physics simulation, robot loading, and visualization.


In [ ]:
from pathlib import Path

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BasicVector,
    Context,
    Diagram,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    ModelInstanceIndex,
    MultibodyPlant,
    Parser,
    RigidTransform,
    Simulator,
    StartMeshcat,
)
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.intro.test_physics_simulation_and_robots import (
    TestPhysicsSimulationDiagramStructure,
    TestPhysicsSimulationFullSystem,
    TestPhysicsSimulationSimpleController,
    TestPhysicsSimulationVerification,
)
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.utils import RenderDiagram

## Meshcat Visualization

Before we start, let us set up Meshcat, our visualization tool.

**Meshcat** is Drake's primary 3D visualization tool that runs in your web browser. It provides real-time interactive visualization of robots, objects, and simulation environments. Meshcat lets you:

- **Visualize 3D robots and scenes** with proper geometry and textures
- **View simulations in real-time** as they run
- **Interact with the 3D scene** (rotate, zoom, pan) 
- **Record videos** for homework submissions
- **Debug your simulations** by seeing what's actually happening

You will find yourself frequently using meshcat to visualize and debug your code!


In [ ]:
# Start meshcat for visualization
meshcat = StartMeshcat()

**Click the link above to open Meshcat in your browser! We will use the same Meshcat window throughout this notebook, so keep it open!**

---

## Part 1: Loading the IIWA14 Robot

First, let's load a real robot! The **IIWA14** is a 7-DOF industrial robot arm from KUKA. Drake includes many robot models that you can load directly from Drake, which is what we will do in this part. You can find a full list of the available Drake models [here](https://github.com/RobotLocomotion/models).

**YOUR TASK:** Load and visualize the IIWA14 robot using Drake's physics engine.

**Key Concepts:**
- `Parser`: Loads robots or assets described with robot description files (like URDF/SDF/MJCF) files into the physics engine
- `MultibodyPlant`: Drake's main physics simulation engine for robots and objects. This will be the backbone for all of our simulations.
- `SceneGraph`: Drake's module for handling everything related to registering robot and object geometries, handling geometry queries, collision detection, etc.
- **Base Welding**: Fixing the robot base to the world frame is important so it doesn't fall!

**Reference:** For this part, we will closely follow the [Drake Simulation example](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/simulation-1ba6290623e34dbbb9d822a2180187c1) from [Chapter 2](https://manipulation.csail.mit.edu/robot.html) from the class textbook - in this part we will follow the **"Simulating the (passive) iiwa"** and **"Visualizing the scene"** sections closely. You should skim through those before you start the implementations below!

In [ ]:
# TODO: Load and visualize the IIWA14 robot


def create_IIWA14_diagram() -> tuple[Diagram, MultibodyPlant, ModelInstanceIndex]:
    # HINT: Look at the Drake Simulation Example above, section "Visualizing the scene"
    # TODO: Create a DiagramBuilder

    # TODO: Add MultibodyPlant and SceneGraph, with a time_step of 1e-4
    # HINT: There is a helper function that does this for you,
    #       it creates both systems and wires them together automatically.

    # TODO: Create a Parser to load robot models
    # NOTE: In the Drake Simulation Example above, the parser is immediately used and
    #       thrown away, whereas we will store it to show you how to use it for adding
    #       multiple models to the plant.

    # TODO: Use the parser to add an IIWA14 model from Drake's model database.
    # HINT: The IIWA14 model in the tutorial has no collision geometry, so let us
    #       instead load a model with collision geometry:
    #       "package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf"
    # HINT: Notice that the parser method `AddModelsFromUrl` returns a list of
    #       references to the loaded models (in this case a list of length 1).
    #       We will need to store the reference to the iiwa model, so make sure
    #       to store the first element of the list as the variable `iiwa`.
    #       (Technically, the reference is of the type `ModelInstanceIndex`,
    #       but don't worry about that for now).

    # TODO: Weld the robot base ("iiwa_link_0") to the world frame

    # TODO: Add MeshcatVisualizer to the builder and connect it to SceneGraph

    # TODO: Finalize the plant (required before simulation)

    # TODO: Build the complete diagram

    # TODO: Return the diagram, plant, and iiwa model instance
    return None, None, None  # Remove this line when you have implemented the function


diagram, plant, iiwa = create_IIWA14_diagram()

In the code above, we used a different model than the model in the tutorial, because we wanted a model with collision geometry. In general, there are many available models for Drake that you can use out of the box. All the models can be found at this [Github repo](https://github.com/RobotLocomotion/models), and the url you provide to Parser follows this format `package://drake_models/{file path in "models" repo}`.

Test your code with the block below:

In [ ]:
Grader.grade_output([TestPhysicsSimulationDiagramStructure], [locals()], "results.json")
Grader.print_test_results("results.json")

**Let us also visualize the block diagram, so you can have a look at the whole setup!**

- Notice that `MultibodyPlant`, `SceneGraph`, and `MeshcatVisualizer` are all systems in the block diagram.
- Make sure to have a look at the connections between the ports of the systems!

In [ ]:
RenderDiagram(diagram, max_depth=1)

Let us also make a default context for the diagram, and have a look at it, just so we have a sense of what is going on!

**Reminder:** Remember that the context stores the "entire state", or all the dynamical information, of everything in the diagram, such as dynamical states of each system, simulation time, etc.

In [ ]:
# HINT: You already learned how to do this in the first notebook!
# TODO: Create a context for the diagram

# TODO: Print the context

Whoa! There is now a lot of information in the context. Let us just print the context for the plant.

(**HINT:** Remember that for a given `system` and a diagram context, we can retrive the system context using `system.GetMyContextFromRoot(diagram_context)`)

- How many (normal) states are there for the plant? Does this match your expectation (think of how many DOFs the IIWA14 has)?

In [ ]:
# TODO: Get the context of the plant from the diagram_context
# HINT: This is important!
#       Use the method plant.GetMyContextFromRoot(diagram_context),
#       similar to what you did for the pendulum in the first notebook!

# TODO: Print the plant context

---

## Part 2: Simulating (just) the IIWA14 

Great job loading the IIWA14 into Drake!

Now, let us set some initial conditions and simulate the system (that so far only involves an IIWA). Make sure to look at the meshcat visualization when you simulate!

**Reference:** For this part, we keep looking at the [Drake Simulation example](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/simulation-1ba6290623e34dbbb9d822a2180187c1).

In [ ]:
# TODO: Implement robot simulation using your create_IIWA14_diagram function


def simulate_IIWA14(
    q0: np.ndarray, simulation_time: float = 3.0, set_target_realtime_rate: bool = True
) -> np.ndarray:
    # HINT: You already learned how to do much of this in the first notebook!

    # TODO: Use your create_IIWA14_diagram function to create the simulation

    # TODO: Create a diagram context

    # HINT: Look at the Simulation example above, section "Visualizing the scene"
    # TODO: Get the plant's context from the diagram context
    # HINT: This is important! The plant context is a subset of the diagram context,
    #       and we only want to modify the plant context to set the initial joint
    #       positions.
    # HINT: Before, we used `GetMyContextFromRoot`, but now we want to modify the
    #       context, so we need to use `GetMyMutableContextFromRoot` (which returns
    #       a mutable context).

    # TODO: Set the initial joint positions of the robot to the given initial joint
    #       positions q0 (in radians)

    # TODO: Set actuator inputs to zero (for passive simulation)
    # HINT: Look at the Drake Simulation example above, and use the same method to set
    #       the actuator inputs to zero.
    # HINT: IIWA14 has 7 joints, so we need to pass a vector of 7 torque values

    # TODO: Create a simulator

    # TODO: Set real-time rate for visualization (so you can see what's happening)
    # if set_target_realtime_rate:
    # HINT: use simulator.set_target_realtime_rate(1.0)

    # TODO: Run the simulation

    # TODO: Return final joint positions
    # HINT: Before, we showed you how to use a logger to record the history of the
    #       system output. Now, we just want to get the final joint positions of the
    #       robot, so we can use the method `plant.GetPositions(plant_context)`
    #       (**after** the simulation is done) to get the final joint positions.

    return np.zeros(7)

Okay! If everything is working as expected, we should now be able to simulate our system. Let us try by running the code below (if everything is working correctly, you should see the IIWA14 swinging passively in your meshcat window).


In [ ]:
# Test your implementation
q_initial = np.array([0, 0, 0, 0, 0, 0, 0])

print("Starting robot simulation...")
print(f"Check your Meshcat window ({meshcat.web_url()}) to see the robot simulation!")
print("     - If the robot is not moving, the simulation already finished.")
print(
    "     - Try running this cell again (or increase the simulation time), with the meshcat window open!"
)
q_final = simulate_IIWA14(
    q_initial, simulation_time=20.0 if running_as_notebook else 0.1
)

print("Robot simulation completed!")
print(f"   Initial joint positions: {q_initial}")
print(f"   Final joint positions: {q_final}")

Okay, that worked. However, the IIWA is just swinging passively. Let us write a simple controller that can at least (almost) keep the IIWA at a desired position! We will implement our (very simple) controller as a `LeafSystem`, as in the first notebook. Note that Drake implements many sophisticated controllers for you, and that you would typically want to use one of those or write a better controller. Here we will write a very simple (and stupid) controller on our own, just for the sake of learning how to connect a custom `LeafSystem` to the `MultibodyPlant`!

In [ ]:
class SimpleController(LeafSystem):
    def __init__(self, gain: float, q_desired: np.ndarray) -> None:
        # TODO: Define a LeafSystem with 14 inputs (IIWA state, i.e. 7 positions + 7 velocities)
        #       and 7 outputs (IIWA joint torques). The output should be
        #       computed by the method `ComputeTorque`.
        # HINT: You know how to do all of this from the first notebook!
        pass

    def ComputeTorque(self, context: Context, output: BasicVector) -> None:
        # TODO: Implement a simple controller with the given gain, that
        #       outputs the torque as `tau = -gain * (q - q_desired)`.
        # HINT: You need to extract the first 7 elements from the state as
        #       the joint positions `q`.
        # HINT: You know how to do the rest of this from the first notebook!
        pass

Run the code below to test your implementation:

In [ ]:
Grader.grade_output([TestPhysicsSimulationSimpleController], [locals()], "results.json")
Grader.print_test_results("results.json")

In [ ]:
# TODO: Load and visualize the IIWA14 robot and connect your controller


def create_IIWA14_diagram_with_controller(
    controller_gain: float, q_desired: np.ndarray
) -> tuple[Diagram, MultibodyPlant, ModelInstanceIndex]:
    # TODO: Build a diagram with the IIWA and your controller. This
    #       should be mostly the same as your create_IIWA14_diagram function,
    #       but with the controller connected to the IIWA.

    # TODO: Create a diagram builder, add a multibody plant and a scene graph to it,
    #       create a parser, load the IIWA, weld the base.

    # TODO: Call plant.Finalize() before you start connecting the controller!

    # TODO: create a controller with the provided function arguments

    # TODO: Connect the state output port for the IIWA from the plant to the input of the controller
    # HINT: Use the `plant.get_state_output_port(iiwa)` to get the state port for the IIWA,
    #       and `controller.{YOUR_INPUT_PORT_NAME}` to get the input port for the controller,
    #       where `iiwa` is the reference to the IIWA model you got from the parser.

    # TODO: Connect the output of the controller to the actuation input port of the plant
    # HINT: Use the `controller.{YOUR_OUTPUT_PORT_NAME}` to get the output port for the controller,
    #       and `plant.get_actuation_input_port(iiwa)` to get the actuation input port for the plant.

    # TODO: Connect the visualizer, build the diagram.
    return None, None, None

Let us print the diagram to make sure that the connections are as expected. Make sure the the output port `iiwa14_state` from the `MultibodyPlant` system is connect to the input port of the controller, and that the output port of the controller is connected to `iiwa14_actuation` in `MultibodyPlant`!

In [ ]:
RenderDiagram(
    create_IIWA14_diagram_with_controller(10.0, np.array([1, 1, 0, 0, 0, 0, 0]))[0],
    max_depth=1,
)

Almost there! Now, just implement the `simulate_IIWA14` function again, but this time use your newly defined function `create_IIWA14_diagram_with_controller`:

In [ ]:
# TODO: Implement robot simulation using your create_IIWA14_diagram_with_controller function


def simulate_IIWA14_with_controller(
    q0: np.ndarray,
    controller_gain: float,
    q_desired: np.ndarray,
    simulation_time=3.0,
    set_target_realtime_rate=True,
) -> np.ndarray:

    return np.zeros(7)

Amazing! Now let us run a simulation. How high do you have to set the gain to make the IIWA keep its starting configuration?

In [ ]:
# Test your implementation
q_initial = np.array([0, 1.0, 0.3, 0.7, 0, 0, 0])

print("Starting robot simulation...")
print(f"Check your Meshcat window ({meshcat.web_url()}) to see the robot simulation!")
print("     - If the robot is not moving, the simulation already finished.")
print(
    "     - Try running this cell again (or increase the simulation time), with the meshcat window open!"
)
q_final = simulate_IIWA14_with_controller(
    q_initial,
    controller_gain=10,
    q_desired=q_initial,
    simulation_time=15 if running_as_notebook else 0.1,
    set_target_realtime_rate=running_as_notebook,
)

print("✅ Robot simulation completed!")
print(f"   Initial joint positions: {q_initial}")
print(f"   Final joint positions: {q_final}")

**NOTE:** You will notice that this controller does not work very well. That is because we are only using a simple proportional controller (a "P-controller"), which only inputs a multiple of the tracking error. The purpose of this exercise is to teach you how to use Drake, but we will soon learn about much better control methods!

### VERIFICATION IN GRADESCOPE

Simulate your system with the following values, and copy/paste the final joint positions of the IIWA14:
- initial positions: $[0.2, 0.2, 0.2, 0, 0, 0, 0]$
- q_desired: $[0, 0, 0, 0, 0, 0, 0]$
- Controller gain: $120.0$
- Simulation time: $10 s$



In [ ]:
# TODO: Simulate the IIWA14 with the specified initial conditions and simulation time

---

## Part 3: Creating Custom Models with SDFormat

Let's learn how to create custom objects for simulation! In the previous two parts, we used a predefined SDFormat(`.sdf`) file to load the IIWA14. We'll now create a simple table from scratch using the same format, then we will use a provided API to generate more complicated assets (your own initials), and show you how to load both into your simulation.

**YOUR TASK:** 
1. Create a simple SDFormat model for a simple table from scratch
2. Use the provided API to generate URDF files for your initials

**Key Concepts:**
- `SDF`, `URDF`, `MJCF`: Three commonly used (and very similar) XML formats to describe robots and objects for simulation
- **Visual Geometry**: What you see in visualization (can be detailed)
- **Collision Geometry**: What physics engine uses for contact (should be simple)
- **Inertial Properties**: Mass and inertia for realistic physics behavior

**Reference:** For this part, we will be using the [Authoring a Multibody Simulation Tutorial](https://github.com/RobotLocomotion/drake/blob/master/tutorials/authoring_multibody_simulation.ipynb) - this part follows the **"Creating custom models"** section closely, where the tutorial shows how to create a cylinder model from an SDFormat string. You should skim through this section before you start the implementation below!


#### Step 1: Write the SDF file for a simple "table" from scratch

SDF, URDF, and MJCF files are not magic, and are in fact quite simple. Here you will write the SDF file for a simple "table" from scratch as an SDF file, showing you how to create your own assets in the future.

In the following cell we will define the SDF string in the code to make the homework cleaner, but usually these are saved to separate `.sdf` files that can be loaded into Drake. 

**WARNING:** The SDFormat is sensitive to indentation, so make sure to refer to the tutorial for how to indent things! You might also find it useful to refer to the [Official SDFormat Documentation](http://sdformat.org/tutorials)


In [ ]:
# TODO: Create a simple SDFormat string for a "table" (a single, flat box will suffice).
# HINT: Follow the cylinder example from the tutorial closely!
# NOTE: We will later use this as a table to place your initials on top of it,
#       so make sure to make it large enough!
# HINT: Some good dimensions are: 2m x 2m x 0.1m
# HINT: You can assume mass 1 kg, and then look up the inertia tensor for a box in the
#       internet (there are plenty of online inertia calculators out there, and it
#       doesn't have to be perfect, as long as it is physically feasible)
table_sdf = """<?xml version="1.0"?>
"""

Great! If everything went as expected, we should now be able to visualize our custom asset. We will visualize it using Drake's built in `ModelVisualizer`, as illustrated in the tutorial (section "Creating Custom Models"). After writing the visualization code below, you should be able to see your table in the same Meshcat window as before!

**Note:** Make sure to click "Open Controls -> Stop Running" in MeshCat when you're done viewing (you will still be able to see the box, even though the visualization has stopped).


In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")
# HINT: Follow the authoring tutorial's example for visualizing the cylinder!
# TODO: Create a ModelVisualizer instance

# TODO: Load your table_sdf string into the visualizer

# TODO: Run the visualizer to see your box in Meshcat
# NOTE: The tutorial uses `Run(loop_once=False)`, but you can just omit the argument.

# Make sure to click "Open Controls -> Stop Running" in MeshCat when you're done viewing

#### Step 2: Create a custom asset with your initials

In this step, you will create a custom asset from your initials. For this we have provided a library for you that generates `.sdf` files for a given letter, but the structure of the underlying files look exactly like the code snippet you wrote above for the simple table!

Here we have provided you with the code you need to run (it might take up to ~30 seconds to finish creating the letter files!). Feel free to change the font, the size and the extrusion depth!


In [ ]:
# Let us save our files to a new folder called `assets` in the `Files`
# section on the left
output_dir = Path("assets/")

# TODO: Insert your initials here!
your_initials = "BPG"

for letter in your_initials:
    create_sdf_asset_from_letter(
        text=letter,
        font_name="DejaVu Sans",
        letter_height_meters=0.2,
        extrusion_depth_meters=0.07,
        output_dir=output_dir / f"{letter}_model",
    )

Great. Now, before we move on, you should have a look at the files that have been generated in the "file" section on the left under the folder named `assets/`, which looks like the following for a general `letter`:

```
{letter}_model/
├── {letter}_parts/
│   ├── convex_piece_000.obj
│   ├── convex_piece_001.obj
│   ├── convex_piece_002.obj
│   └── ... (more convex mesh files)
├── {letter}.obj
└── {letter}.sdf
```

For instance, for the letter `A`, it looks like:

```
A_model/
├── A_parts/
│   ├── convex_piece_000.obj
│   ├── convex_piece_001.obj
│   ├── convex_piece_002.obj
│   └── ... (more convex mesh files)
├── A.obj
└── A.sdf
```

The `.obj` file described the overall 3D mesh of the letter, and the `convex_piece_XXX.obj` files describe convex parts of the mesh for a letter: these are just text files that contain a list of vertices and faces. Try to open open one of them and see for yourself (click the "Open in raw mode" button in Deepnote to see the textfile!) 

The `.sdf` file contains the asset description for each individual letter, which links to the overall `.obj` file for the visual geometry, and the convex `.obj` files for the collision geometry (where each mesh part must be convex, which is why we generate multiple parts for each letter!).


Okay, great! Now that we have generated the letters, let us use Drake's `ModelVisualizer` and have a look at them! Fill in the missing code below, and make sure that the letters you generated look good.

**NOTE**: By clicking on `Scene > drake` you can enable visualization of `inertia` and `proximity` (collision geometry), which lets you see the physical properties of the asset too. This can be very handy when debugging!

In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")

# TODO: Change this to visualize the individual letters in your initials
letter_to_visualize = "P"  # One at a time!

# This is the path to the SDF file for the letter
letter_path = f"assets/{letter_to_visualize}_model/{letter_to_visualize}.sdf"

# TODO: Define a ModelVisualizer instance
# HINT: You know how to do this by now!

# TODO: Load the letter into the visualizer using the parser
# HINT: You can just pass the path to the letter file to the parser

# TODO: Run the visualizer

---

## Part 4: Complete Simulation - Robot + Your Custom Asset

**The Grand Finale!** Now we'll combine everything you've learned to create a complete simulation with both the IIWA14 robot and your custom initials asset!

**YOUR TASK:** Build a physics simulation containing:
1. **IIWA14 robot**: welded to the world frame as before.
2. **Your custom table**: welded to the world frame near your robot (but not penetrating it!)
3. **Your custom initials**: placed on top of your custom table.

You now have all the tools you need to implement this yourself! Use the patterns you've learned from the previous parts to complete the notebook. When you are done, take a screen recording (must be an `.mp4` file with size below 500 MB) of the simulation and upload it to Gradescope.

**Final Goal:** Use a screen recording tool to record a video of your simulated scene. The video should show your initials falling onto the table, which is welded in place close to the robot (but not penetrating it).

**Hint**: `WeldFrames` takes a third argument a `RigidTransform` (which you will learn about later in the class). For now, you can just pass in a `(x,y,z)`-position with `RigidTransform([x,y,z])` to weld frames with an offset. (As always, if you are unsure about a function signature take a look at the [official documentation](https://drake.mit.edu/pydrake/pydrake.multibody.plant.html))


In [ ]:
# Before you start, make sure to run the following cell to clear the meshcat window!
meshcat.Delete()
meshcat.DeleteAddedControls()

In [ ]:
# TODO: Your implementation here!


def simulate_full_system(
    initial_iiwa_positions: np.ndarray,
    initial_letter_poses: list[RigidTransform],
    table_pose: RigidTransform,
    simulation_time: float = 15.0,
) -> None:
    # TODO: Create a diagram builder and add a multibody plant and a scene graph to it

    # TODO: Load the IIWA14 robot and weld the base to the world frame

    # TODO: Load the table and weld the base to the world frame.
    #       Don't forget to set the pose of the table by passing `table_pose` to the `WeldFrames`
    #       method as the third argument!

    # TODO: Load the letters in your initials
    # HINT: Remember to use 'assets/...' to access the letters, as we did above!

    # TODO: Add a visualizer to the builder

    # TODO: Finalize the plant and build the diagram

    # TODO: Create a diagram context and a plant context

    # TODO: Set the positions of the IIWA14 robot joints.
    # HINT: Since we now have multiple free objects in the scene, we need specify
    #       that we are setting the positions of the IIWA14 robot joints:
    # plant.SetPositions(plant_context, iiwa, initial_iiwa_positions)

    # TODO: Load the letters and set their initial poses.
    # HINT: To set the poses of the letters, we need some new functionality.
    #       This is because the letters are free-floating bodies, and we need to set their poses,
    #       which includes both positions and orientations.
    #
    #       First, we need to get the RigidBody associated with the letter, and then set its pose
    #       using the SetFreeBodyPose method, which accepts a RigidTransform as an argument.
    #
    #       Here is an example of how to do this, for the letter B:
    #       (Important: notice that we first load the letter, then finalize the plant, and then set the pose)
    #
    #               ```
    #               B_letter = parser.AddModels(f"assets/B_model/B.sdf")[0]
    #               ...
    #               plant.Finalize()
    #               ...
    #               body = plant.GetRigidBodyByName("B_body_link", B_letter)
    #               plant.SetFreeBodyPose(plant_context, body, initial_letter_poses[0])
    #               ```
    #
    #       (Note the name "B_body_link", which is hardcoded in the SDF for each letter generated by our
    #       letter generation function.)

    #       Now use the same pattern to set the poses for your letters!

    # TODO: Fix the actuator inputs to zero for the plant

    # TODO: Create a simulator and run the simulation for the given `simulation_time`
    pass

Run the code below to visualize your simulation!

**Tip:** If you ever want to download a recording of your simulation as a `.html` file, append `/download` to the meshcat URL in your browser. Go ahead and try it for your simulation below!

In [ ]:
print(f"Meshcat URL: {meshcat.web_url()}")

# Notice that we have wrapped your function in a `meshcat.StartRecording()` and `meshcat.StopRecording()` block,
# followed by a `meshcat.PublishRecording()` call. This is a convenient way to make it possible to replay the
# simulation in meshcat after the simulation ends!
meshcat.StartRecording()
simulate_full_system(
    initial_iiwa_positions=np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0]),
    initial_letter_poses=[
        # You can add rotations to the RigidTransform if you want, but we will learn more about that later!
        # The letters should be placed such that they are not in collision with each other, and such that
        # they fall onto the table.
        RigidTransform([0.7, 0.0, 1.0]),
        RigidTransform([0.9, 0.0, 1.0]),
        RigidTransform([1.1, 0.0, 1.0]),
    ],
    table_pose=RigidTransform(
        [0.5, 0.0, -0.05]
    ),  # this is a reasonable position for a 2m x 2m x 0.05m table, but feel free to change it!
    simulation_time=5.0 if running_as_notebook else 0.1,
)
meshcat.StopRecording()
meshcat.PublishRecording()

# VERIFICATION IN GRADESCOPE
Congratulations! You are all done with this notebook. Now play the simulation back in your browser, and upload a screen recording of it in Gradescope (remember to make sure its an `mp4` file with size less than 500 MB).

---

# Congratulations!

You have successfully completed **Notebook 2: Physics Simulation and Robot Loading**!

### Summary

In this notebook, you learned to:

- Visualize robots, objects, and simulations using Meshcat
- Load real robots into Drake using `Parser` and robot description files (we worked only with SDF, but URDF and MJCF are pretty similar!)
- Understand Drake's physics simulation engine (`MultibodyPlant`) and geometry engine (`SceneGraph`)
- Create custom 3D assets and learn URDF/SDF authoring
- Build complete robot simulations with custom objects in Drake

You now have the foundational skills to work with Drake's physics simulation capabilities and create custom robotic environments. These skills will be essential for the more advanced topics later in the class, as well as for your class project!

**Next:** In Notebook 3, you will learn how to quickly assemble complex scenes using a `scenario.yaml` file and `HardwareStation`.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e32ce10-e406-4026-abfb-00be78c478cb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>